In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
!pip install mlflow dagshub
import dagshub
import mlflow
dagshub.init(repo_owner='skara-21', repo_name='Assignment2_Fraud_Detection', mlflow=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=04aeed45-6729-407b-82a4-2e5ca8e8fee1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a2cac739c19289d157df183324ff72d8d6a9af82ecb8463d3f7d0115b465c8d3




Output()

Accessing as skara-21

Initialized MLflow to track repo "skara-21/Assignment2_Fraud_Detection"

Repository skara-21/Assignment2_Fraud_Detection initialized!

In [3]:
train_id=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

In [4]:
train=pd.merge(train_trans,train_id,on='TransactionID',how='left')

In [5]:
del train_trans,train_id

In [6]:
cat_columns=['ProductCD','addr1','addr2','P_emaildomain',
             'R_emaildomain','DeviceType','DeviceInfo',
            'user_has_ever_been_fraud','userId']
cat_columns+=['card'+str(i) for i in range(1,7)]
cat_columns+=['M'+str(i) for i in range(1,10)]
cat_columns+=['id_'+str(i) for i in range(12,39)]

num_columns=[col for col in train.columns.tolist() if col not in cat_columns and col != 'isFraud']

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

In [8]:
class DropMissing(BaseEstimator, TransformerMixin):
    def __init__(self,threshold=90):
        self.to_remove=None

    def fit(self,X, y=None):
        cols_with_missing = [col for col in X.columns
                         if X[col].isnull().any()]
        to_remove=[]
        for col in cols_with_missing:
            null_percentile=(X[col].isnull().mean())*100
            if null_percentile>90: to_remove.append(col)
        self.to_remove=to_remove
        print('dropped missing')
        return self

    def transform(self,X):
        if self.to_remove==None:
            raise Error('you must run fit method first to drop columns with missing values')
            return X
        else:
            return X.drop(self.to_remove,axis=1)

In [9]:
class FillNaN(BaseEstimator, TransformerMixin):
    def __init__(self,cat_columns,num_columns,threshold=90):
        self.to_remove=None
        self.num_columns=num_columns
        self.cat_columns=cat_columns
    def fit(self,X,y=None):
        X_cp=X.copy()
        to_remove={}
        for col in self.num_columns:
            if col in X_cp.columns:
                mode_val = X_cp[col].mode()[0]
                X_cp.fillna({col:mode_val}, inplace=True)
                to_remove[col]=mode_val
        self.to_remove = to_remove
        for col in self.cat_columns:
            X_cp.fillna({col:'missing'}, inplace=True)

        print('filled nan')
        return self

    def transform(self,X):
        if self.to_remove==None:
            raise Error('you must run fit method first to fill NaN values')
        else:
            X_cp=X.copy()
            for col,val in self.to_remove.items():
                X_cp.fillna({col:val}, inplace=True)
            return X_cp

In [10]:
from scipy.stats.mstats import winsorize
class RemoveOutliers(BaseEstimator, TransformerMixin):
    def __init__(self,num_columns,threshold=[0.01,0.01]):
        self.num_columns=num_columns
        self.threshold=threshold
    def fit(self,X,y=None):
        for col in self.num_columns:
            if col in X.columns:
                X[col]=winsorize(X[col], limits=self.threshold)

        print('removed outliers')
        return self
    def transform(self,X):
        for col in self.num_columns:
            if col in X.columns:
                X[col]=winsorize(X[col], limits=self.threshold)
        return X

In [11]:
# First install compatible versions
!pip install scikit-learn==1.0.2 category_encoders==2.2.2

# Then modify your BinaryNonBinaryEncoder class as follows:
from category_encoders import TargetEncoder  # Using TargetEncoder instead of WOEEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class BinaryNonBinaryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, binary_threshold=2):
        self.binary_threshold = binary_threshold
        self.binary_cols_ = None
        self.non_binary_cols_ = None
        self.encoder_ = None
        self.ohe_columns_ = None
        
    def fit(self, X, y):
        # Convert to DataFrame if not already
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        
        self.binary_cols_ = [col for col in X.columns 
                           if X[col].nunique() <= self.binary_threshold]
        self.non_binary_cols_ = [col for col in X.columns 
                               if X[col].nunique() > self.binary_threshold]
        
        if self.non_binary_cols_:
            self.encoder_ = TargetEncoder(
                cols=self.non_binary_cols_,
                handle_unknown='value',
                handle_missing='value'
            )
            self.encoder_.fit(X[self.non_binary_cols_], y)
        
        if self.binary_cols_:
            dummy_df = pd.get_dummies(X[self.binary_cols_], drop_first=True, dtype=int)
            self.ohe_columns_ = dummy_df.columns.tolist()

        print('encoded')
        return self
        
    def transform(self, X):
        # Convert to DataFrame if not already
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        
        if self.non_binary_cols_ and self.encoder_:
            X[self.non_binary_cols_] = self.encoder_.transform(X[self.non_binary_cols_])
        
        if self.binary_cols_:
            dummy_df = pd.get_dummies(X[self.binary_cols_], drop_first=True, dtype=int)
            
            # Handle missing columns
            missing_cols = set(self.ohe_columns_) - set(dummy_df.columns)
            for col in missing_cols:
                dummy_df[col] = 0
            
            # Ensure correct column order
            dummy_df = dummy_df.reindex(columns=self.ohe_columns_, fill_value=0)
            
            X = pd.concat([
                X.drop(columns=self.binary_cols_),
                dummy_df
            ], axis=1)
        
        return X
    
    def fit_transform(self, X, y=None, **fit_params):
        if y is None:
            raise ValueError("y cannot be None for encoding")
        return self.fit(X, y).transform(X)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.4 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [12]:
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class XGBRFE(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 n_features_to_select=150,
                 n_estimators=100,
                 max_depth=6,
                 learning_rate=0.05,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 n_jobs=-1,
                 random_state=42,
                 verbosity=0):
        self.n_features_to_select = n_features_to_select
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbosity = verbosity
        self.selected_features_ = None
        self.support_ = None
        self.ranking_ = None

    def fit(self, X, y):
        print('Starting optimized feature selection...')
        
        estimator = XGBRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            learning_rate=self.learning_rate,
            subsample=self.subsample,
            colsample_bytree=self.colsample_bytree,
            n_jobs=self.n_jobs,
            random_state=self.random_state,
            verbosity=self.verbosity,
            tree_method='hist',
            booster='gbtree',
            objective='reg:squarederror'
        )
        
        if X.shape[1] > 200:
            print('Phase 1: Quick elimination of worst features')
            initial_selector = RFE(
                estimator=estimator,
                n_features_to_select=min(200, X.shape[1]),
                step=max(1, X.shape[1]//20)
            )
            initial_selector.fit(X, y)
            X = X.loc[:, initial_selector.support_]
        
        print('Phase 2: Final feature selection')
        final_selector = RFE(
            estimator=estimator,
            n_features_to_select=self.n_features_to_select,
            step=1
        )
        final_selector.fit(X, y)
        
        if hasattr(X, 'columns'):
            self.selected_features_ = X.columns[final_selector.support_].tolist()
        self.support_ = final_selector.support_
        self.ranking_ = final_selector.ranking_
        
        print(f'Completed. Selected {len(self.selected_features_)} features.')
        return self

    def transform(self, X):
        if self.selected_features_ is None:
            raise RuntimeError("Must fit transformer before transforming")
            
        if isinstance(X, pd.DataFrame):
            return X[self.selected_features_]
        return X[:, self.support_]

    def get_support(self, indices=False):
        if indices:
            return np.where(self.support_)[0]
        return self.support_

In [13]:
class CorrelationRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.85):
        self.threshold = threshold
        self.cols_to_drop_ = None
        
    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            corr_matrix = X.corr().abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
            self.cols_to_drop_ = [col for col in X.columns 
                                if any(upper[col] > self.threshold)]

        print('removed correlations')
        return self
        
    def transform(self, X):
        if self.cols_to_drop_ is None:
            raise RuntimeError("Must fit transformer before transforming data")
        return X.drop(columns=self.cols_to_drop_)

In [14]:
print('done')

done


In [15]:
!pip install --upgrade scikit-learn==1.0.2 imbalanced-learn
!pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

  Using cached scikit-learn-1.0.2.tar.gz (6.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0


In [16]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Training

In [17]:
import mlflow
import mlflow.sklearn

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_score, 
    recall_score, classification_report, f1_score
)
from sklearn.ensemble import RandomForestClassifier

mlflow.set_tracking_uri('https://dagshub.com/skara-21/Assignment2_Fraud_Detection.mlflow')
mlflow.set_experiment("RandomForest_Training")

X = train.drop('isFraud', axis=1)
y = train['isFraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

fraud_pipeline = Pipeline([
    ('drop_missing', DropMissing(threshold=90)),
    ('fill_na', FillNaN(cat_columns=cat_columns, num_columns=num_columns)),
    ('remove_outliers', RemoveOutliers(num_columns=num_columns, threshold=[0.01, 0.01])),
    ('binary_encoder', BinaryNonBinaryEncoder(binary_threshold=2)),
    ('correlation_remover', CorrelationRemover(threshold=0.85)),
    ('feature_selector', XGBRFE(n_features_to_select=int(len(X_train.columns)*0.8))),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        class_weight='balanced',
        random_state=42
    ))
])

with mlflow.start_run(run_name="RandomForest_Fraud_Detection"):
    mlflow.log_params({
        "preprocessing_steps": [step[0] for step in fraud_pipeline.steps],
        "classifier_type": "RandomForest",
        "class_weight": "balanced",
        "feature_selection": "XGBRFE"
    })

    fraud_pipeline.fit(X_train, y_train)
    
    y_pred_proba = fraud_pipeline.predict_proba(X_val)[:, 1]
    y_pred = fraud_pipeline.predict(X_val)
    
    f1 = f1_score(y_val, y_pred)

    mlflow.log_metrics({
        "val_auc": roc_auc_score(y_val, y_pred_proba),
        "val_ap": average_precision_score(y_val, y_pred_proba),
        "val_f1": f1,
        "fraud_precision": precision_score(y_val, y_pred),
        "fraud_recall": recall_score(y_val, y_pred, pos_label=1)
    })

    imbalance_report = classification_report(y_val, y_pred, output_dict=True)
    mlflow.log_dict(imbalance_report, "imbalance_classification_report.json")


    mlflow.sklearn.log_model(
        fraud_pipeline,
        "fraud_detection_pipeline",
        registered_model_name="Fraud_RF_Pipeline"
    )

    print(f"Training complete. F1 Score: {f1:.4f}")

dropped missing
filled nan
removed outliers
encoded


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


removed correlations
Starting optimized feature selection...
Phase 1: Quick elimination of worst features
Phase 2: Final feature selection
Completed. Selected 200 features.


2025/04/29 09:47:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Fraud_RF_Pipeline' already exists. Creating a new version of this model...
2025/04/29 09:47:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Fraud_RF_Pipeline, version 3
Created version '3' of model 'Fraud_RF_Pipeline'.


Training complete. F1 Score: 0.0099
🏃 View run RandomForest_Fraud_Detection at: https://dagshub.com/skara-21/Assignment2_Fraud_Detection.mlflow/#/experiments/1/runs/fa7311aabc00458f94d1870164eaaf33
🧪 View experiment at: https://dagshub.com/skara-21/Assignment2_Fraud_Detection.mlflow/#/experiments/1
